# 🏎️ F1 2026 Technical Regulations Impact Simulator

## Complete Analysis Pipeline with Track-by-Track Visualizations

This notebook provides a comprehensive analysis of how the **2026 F1 Technical Regulations** will impact race performance across different circuits.

### Key 2026 Regulation Changes Analyzed:
- **ERS Power Split**: 50% electric power (up from 15%)
- **Active Aerodynamics**: Movable aero elements for efficiency
- **Weight Reduction**: 768kg minimum (down from 798kg)
- **Tire Changes**: New 18-inch low-profile tires
- **Fuel Flow Limits**: Reduced flow rate with efficiency focus

---

## 1. Setup & Configuration

In [2]:
# Core imports
from pathlib import Path
import json
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Set project root (local system)
project_root = Path.cwd()

# If we're in notebooks/, go up one level
if project_root.name == 'notebooks' and (project_root.parent / 'src').exists():
    project_root = project_root.parent
elif not (project_root / 'src').exists() and (project_root.parent / 'src').exists():
    project_root = project_root.parent

# Add to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
if str(project_root / 'src') not in sys.path:
    sys.path.insert(0, str(project_root / 'src'))

# Create outputs directory
charts_dir = project_root / 'outputs' / 'comparison_charts'
charts_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Project root: {project_root}")
print(f"✅ src path: {project_root / 'src'}")
print(f"✅ Charts directory: {charts_dir}")

✅ Project root: e:\5thsem\AIML\f1-2026-simulator
✅ src path: e:\5thsem\AIML\f1-2026-simulator\src
✅ Charts directory: e:\5thsem\AIML\f1-2026-simulator\outputs\comparison_charts


In [4]:
# Data science stack
import numpy as np
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# ML
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# Detect runtime environment for renderer selection
IN_COLAB = "google.colab" in sys.modules

# Set Plotly renderer based on environment
if IN_COLAB:
    pio.renderers.default = 'colab'
else:
    try:
        import nbformat
        pio.renderers.default = 'notebook'
    except ImportError:
        pio.renderers.default = 'browser'

print(f"✅ NumPy: {np.__version__}, Pandas: {pd.__version__}")
print(f"✅ Plotly renderer: {pio.renderers.default}")

✅ NumPy: 1.26.4, Pandas: 2.3.3
✅ Plotly renderer: browser


In [5]:
# Import project modules
try:
    from src.features import engineer_features
    from src.monte_carlo import MonteCarloSimulator, SimulationConfig
    from src.regulation_transform import apply_2026_regulations, REGULATION_MULTIPLIERS
    from src.visualization import (
        create_team_impact_heatmap,
        create_monte_carlo_violins,
        draw_circuit_before_after,
        create_track_regulation_dashboard,
        create_factor_impact_by_track_type,
        create_position_change_waterfall,
        create_track_comparison_radar,
        create_grid_of_track_impacts,
        create_regulation_impact_summary_chart
    )
    from src.data_loader import load_circuits_metadata
    print("✅ All project modules imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print(f"   sys.path: {sys.path[:3]}")
    print(f"   project_root: {project_root}")
    print(f"   src exists: {(project_root / 'src').exists()}")
    
    # List src contents for debugging
    if (project_root / 'src').exists():
        print(f"   src contents: {list((project_root / 'src').iterdir())}")
    raise

✅ All project modules imported successfully!


## 2. Data Loading

Load race data from the processed CSV file containing 2022-2025 season results.

In [6]:
# Load race data
cache_file = project_root / "data" / "processed" / "f1_2022_2025.csv"

if cache_file.exists():
    dataset = pd.read_csv(cache_file)
    print(f"✅ Loaded {len(dataset)} race results from {cache_file.name}")
else:
    # Try to use FastF1 if available and cache file doesn't exist
    try:
        from src.data_loader import load_f1_data
        seasons = [2022, 2023, 2024, 2025]
        print(f"📡 Fetching data for seasons {seasons} via FastF1...")
        dataset = load_f1_data(seasons, cache_path=cache_file)
        print(f"✅ Fetched and cached data for seasons {seasons}")
    except Exception as e:
        raise FileNotFoundError(
            f"Data file not found at {cache_file} and FastF1 fetch failed: {e}\n"
            f"Please run main.py first to generate the data file, or ensure FastF1 is installed."
        )

# Display data summary
print(f"\n📊 Dataset Summary:")
print(f"   - Seasons: {sorted(dataset['season'].unique().tolist())}")
print(f"   - Total races: {dataset[['season', 'round']].drop_duplicates().shape[0]}")
print(f"   - Unique drivers: {dataset['driver_name'].nunique()}")
if 'circuit' in dataset.columns:
    print(f"   - Unique circuits: {dataset['circuit'].nunique()}")

dataset.head()

✅ Loaded 1838 race results from f1_2022_2025.csv

📊 Dataset Summary:
   - Seasons: [2022, 2023, 2024, 2025]
   - Total races: 92
   - Unique drivers: 32
   - Unique circuits: 26


,driver_number,driver_code,driver_name,team_name,position,grid,points,status,dnf_flag,season,...,track_temp_c,humidity_pct,pressure_hpa,rainfall_mm,wind_speed_kph,pit_stop_count,compound_changes,compound_sequence,stint_lengths,avg_lap_time_seconds
0,16,LEC,Charles Leclerc,Ferrari,1,1,26.0,Finished,0,2022,...,28.610429,29.490798,1010.38589,0.0,0.304908,3.0,1.0,"[""SOFT"", ""MEDIUM""]","[15, 16, 15, 11]",100.697709
1,55,SAI,Carlos Sainz,Ferrari,2,3,18.0,Finished,0,2022,...,28.610429,29.490798,1010.38589,0.0,0.304908,3.0,1.0,"[""SOFT"", ""MEDIUM""]","[14, 19, 11, 13]",102.792667
2,44,HAM,Lewis Hamilton,Mercedes,3,5,15.0,Finished,0,2022,...,28.610429,29.490798,1010.38589,0.0,0.304908,3.0,2.0,"[""SOFT"", ""HARD"", ""MEDIUM""]","[11, 16, 17, 13]",102.864193
3,63,RUS,George Russell,Mercedes,4,9,12.0,Finished,0,2022,...,28.610429,29.490798,1010.38589,0.0,0.304908,3.0,2.0,"[""SOFT"", ""HARD"", ""MEDIUM""]","[15, 18, 12, 12]",102.040107
4,20,MAG,Kevin Magnussen,Haas F1 Team,5,7,10.0,Finished,0,2022,...,28.610429,29.490798,1010.38589,0.0,0.304908,3.0,1.0,"[""SOFT"", ""MEDIUM""]","[14, 20, 12, 11]",102.953298


In [7]:
# Load circuit metadata for track-specific analysis
try:
    circuit_metadata = load_circuits_metadata()
    print(f"✅ Loaded metadata for {len(circuit_metadata)} circuits")
    display(circuit_metadata.head(10))
except Exception as e:
    print(f"⚠️ Could not load circuit metadata: {e}")
    print("   Using default empty DataFrame")
    circuit_metadata = pd.DataFrame()

Circuit metadata CSV not found at E:\5thsem\AIML\f1-2026-simulator\data\raw\circuits.csv. Using bundled defaults.


✅ Loaded metadata for 19 circuits


,circuit_name,country,track_type,corners,straight_fraction,overtaking_difficulty,circuit_key
0,Bahrain,Bahrain,night-street,15,0.58,3,bahrain
1,Jeddah,Saudi Arabia,street,27,0.63,4,jeddah
2,Melbourne,Australia,street,14,0.42,3,melbourne
3,Imola,Italy,high-downforce,19,0.41,4,imola
4,Monaco,Monaco,street,19,0.25,5,monaco
5,Barcelona,Spain,balanced,16,0.47,3,barcelona
6,Montreal,Canada,semi-street,14,0.54,2,montreal
7,Silverstone,United Kingdom,high-speed,18,0.52,2,silverstone
8,Spielberg,Austria,high-speed,10,0.64,2,spielberg
9,Budapest,Hungary,high-downforce,14,0.34,4,budapest


## 3. Feature Engineering

Create 25 curated features for the ML model including driver form, track characteristics, and regulation parameters.

In [8]:
# Engineer features
features = engineer_features(dataset)

# Identify feature columns (exclude metadata)
meta_cols = {"position", "driver_name", "team_name", "season", "round", "event_name"}
feature_cols = [c for c in features.columns if c not in meta_cols]

# Calculate feature means for imputation
feature_means = features[feature_cols].mean()

print(f"✅ Engineered {len(feature_cols)} features")
print(f"\n📋 Feature columns:")
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {col}")

✅ Engineered 25 features

📋 Feature columns:
    1. avg_pos_last5
    2. points_last5
    3. dnf_count_last5
    4. grid_position
    5. grid_vs_race_delta
    6. track_type_index
    7. corners
    8. straight_fraction
    9. overtaking_difficulty
   10. rain_probability
   11. track_temperature
   12. wind_speed
   13. pit_stops_count
   14. tire_compound_change_count
   15. fuel_efficiency_rating
   16. power_ratio
   17. aero_coeff
   18. weight_ratio
   19. tire_grip_ratio
   20. fuel_flow_ratio
   21. team_consistency_score
   22. driver_aggressiveness_index
   23. season_year
   24. round_number
   25. season_phase


In [9]:
# Feature statistics visualization
feature_std = features[feature_cols].std()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=feature_means.index,
    y=feature_means.values,
    name="Mean",
    marker_color="steelblue"
))
fig.add_trace(go.Scatter(
    x=feature_std.index,
    y=feature_std.values,
    name="Std Dev",
    mode="markers",
    marker=dict(color="red", size=10, symbol="diamond")
))
fig.update_layout(
    title="📊 Feature Statistics Overview",
    xaxis_tickangle=-45,
    height=500,
    showlegend=True
)
fig.show()

## 4. Model Training

Train an XGBoost regressor to predict race positions using the engineered features.

In [10]:
# Prepare data
X = features[feature_cols].fillna(feature_means)
y = features["position"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist"
)

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n🎯 Model Performance:")
print(f"   - MAE: {mae:.2f} positions")
print(f"   - R²: {r2:.3f}")

# Cross-validation with TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(model, X, y, cv=tscv, scoring="neg_mean_absolute_error")
print(f"   - CV MAE: {-cv_scores.mean():.2f} ± {cv_scores.std():.2f}")


🎯 Model Performance:
   - MAE: 0.34 positions
   - R²: 0.991
   - CV MAE: 0.59 ± 0.22


In [11]:
# Feature importance visualization
importance = pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=True)

fig = go.Figure(go.Bar(
    x=importance.values[-15:],
    y=importance.index[-15:],
    orientation='h',
    marker_color='steelblue'
))
fig.update_layout(
    title="🔍 Top 15 Most Important Features",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=500
)
fig.show()

## 5. Monte Carlo Simulations Per Track

Run Monte Carlo simulations for **each unique race**, comparing current regulations vs 2026 regulations.

In [12]:
# Display 2026 Regulation Multipliers
print("⚙️ 2026 Regulation Multipliers:")
print("="*50)
for category, multipliers in REGULATION_MULTIPLIERS.items():
    print(f"\n📦 {category.replace('_', ' ').title()}:")
    for param, value in multipliers.items():
        change = ((value - 1) * 100)
        direction = "↑" if change > 0 else "↓" if change < 0 else "→"
        print(f"   {param}: {value:.2f}x ({direction} {abs(change):.0f}%)")

⚙️ 2026 Regulation Multipliers:

📦 Hybrid Power:
   power_ratio: 3.33x (↑ 233%)

📦 Boost Mode:
   power_ratio: 1.25x (↑ 25%)
   fuel_efficiency_rating: 1.05x (↑ 5%)
   overtake_power_boost: 1.15x (↑ 15%)
   ers_deployment_flexibility: 1.40x (↑ 40%)

📦 Chassis:
   weight_ratio: 0.96x (↓ 4%)

📦 Tyres:
   tire_grip_ratio: 0.94x (↓ 6%)

📦 Fuel:
   fuel_flow_ratio: 0.75x (↓ 25%)
   fuel_efficiency_rating: 1.15x (↑ 15%)


In [13]:
# Configure Monte Carlo simulator
config = SimulationConfig(
    n_simulations=2000,
    driver_form_sigma=0.05,
    weather_sigma=0.10,
    strategy_delta=0.10,
    random_seed=42
)

simulator = MonteCarloSimulator(model, feature_cols, config)

print(f"✅ Monte Carlo Simulator configured:")
print(f"   - Simulations per race: {config.n_simulations}")
print(f"   - Driver form sigma: {config.driver_form_sigma}")
print(f"   - Weather sigma: {config.weather_sigma}")

✅ Monte Carlo Simulator configured:
   - Simulations per race: 2000
   - Driver form sigma: 0.05
   - Weather sigma: 0.1


In [14]:
# Run simulations for each race
grouped = features.groupby(["season", "round"], sort=True)
simulation_results = {}

print("\n🏁 Running Monte Carlo Simulations...")
print("="*60)

for (season, round_number), race_data in grouped:
    race_features = race_data[feature_cols].fillna(feature_means)
    drivers = race_data["driver_name"].tolist()
    
    # Current regulations simulation
    current_stats = simulator.run(race_features, drivers)
    
    # 2026 regulations simulation
    future_frame = apply_2026_regulations(race_features)
    future_stats = simulator.run(future_frame, drivers)
    
    # Store results
    race_key = f"{season}_R{round_number:02d}"
    event_name = race_data["event_name"].iloc[0] if "event_name" in race_data.columns else race_key
    
    simulation_results[race_key] = {
        "event_name": event_name,
        "current": current_stats,
        "2026": future_stats
    }
    
    print(f"✓ {race_key}: {event_name[:35]:35s}")

print(f"\n✅ Completed {len(simulation_results)} race simulations!")


🏁 Running Monte Carlo Simulations...
✓ 2022_R01: Bahrain Grand Prix                 
✓ 2022_R02: Saudi Arabian Grand Prix           
✓ 2022_R03: Australian Grand Prix              
✓ 2022_R04: Emilia Romagna Grand Prix          
✓ 2022_R05: Miami Grand Prix                   
✓ 2022_R06: Spanish Grand Prix                 
✓ 2022_R07: Monaco Grand Prix                  
✓ 2022_R08: Azerbaijan Grand Prix              
✓ 2022_R09: Canadian Grand Prix                
✓ 2022_R10: British Grand Prix                 
✓ 2022_R11: Austrian Grand Prix                
✓ 2022_R12: French Grand Prix                  
✓ 2022_R13: Hungarian Grand Prix               
✓ 2022_R14: Belgian Grand Prix                 
✓ 2022_R15: Dutch Grand Prix                   
✓ 2022_R16: Italian Grand Prix                 
✓ 2022_R17: Singapore Grand Prix               
✓ 2022_R18: Japanese Grand Prix                
✓ 2022_R19: United States Grand Prix           
✓ 2022_R20: Mexico City Grand Prix             
✓ 

In [15]:
# Save simulation results
output_dir = project_root / "outputs"
output_dir.mkdir(exist_ok=True)

results_path = output_dir / "monte_carlo_results.json"
with open(results_path, "w", encoding="utf-8") as f:
    json.dump(simulation_results, f, indent=2)

print(f"✅ Results saved to {results_path}")

✅ Results saved to e:\5thsem\AIML\f1-2026-simulator\outputs\monte_carlo_results.json


## 6. Track-by-Track Regulation Impact Visualizations

Comprehensive visualizations showing how 2026 regulations affect different track types.

### 6.1 Regulation Factor Impact by Track Type

In [16]:
# Factor impact by track type
fig = create_factor_impact_by_track_type(circuit_metadata)
fig.show()

### 6.2 Team Impact Heatmap

In [17]:
# Team impact heatmap
fig = create_team_impact_heatmap(simulation_results)
fig.update_layout(height=600, width=1200)
fig.show()

### 6.3 Cumulative Position Impact (Waterfall Chart)

In [18]:
# Position change waterfall
fig = create_position_change_waterfall(simulation_results)
fig.show()

### 6.4 Track-by-Track Position Impact Grid

In [19]:
# Grid of track impacts
fig = create_grid_of_track_impacts(simulation_results, max_tracks=12)
fig.show()

### 6.5 Regulation Impact Summary

In [20]:
# Overall summary chart
fig = create_regulation_impact_summary_chart(simulation_results)
fig.show()

## 7. Key Insights & Analysis

In [21]:
# Aggregate analysis
print("\n" + "="*60)
print("📊 2026 REGULATION IMPACT SUMMARY")
print("="*60)

# Calculate overall statistics
all_changes = []
driver_impacts = {}

for key, race_data in simulation_results.items():
    current = pd.DataFrame(race_data["current"]).T
    future = pd.DataFrame(race_data["2026"]).T
    
    position_change = current["mean"] - future["mean"]
    all_changes.extend(position_change.tolist())
    
    for driver in position_change.index:
        if driver not in driver_impacts:
            driver_impacts[driver] = []
        driver_impacts[driver].append(position_change[driver])

# Overall statistics
all_changes = np.array(all_changes)
print(f"\n🏎️ Overall Position Impact:")
print(f"   - Average change: {all_changes.mean():+.3f} positions")
print(f"   - Standard deviation: {all_changes.std():.3f}")
print(f"   - Max improvement: {all_changes.max():.3f}")
print(f"   - Max regression: {all_changes.min():.3f}")
print(f"   - Drivers improved: {(all_changes > 0).sum()} ({(all_changes > 0).mean()*100:.1f}%)")
print(f"   - Drivers regressed: {(all_changes < 0).sum()} ({(all_changes < 0).mean()*100:.1f}%)")

# Driver summary
driver_avg = {d: np.mean(v) for d, v in driver_impacts.items()}
sorted_drivers = sorted(driver_avg.items(), key=lambda x: x[1], reverse=True)

print(f"\n🏆 Drivers Most Helped by 2026 Regulations:")
for driver, impact in sorted_drivers[:5]:
    print(f"   {impact:+.3f} positions: {driver}")

print(f"\n⚠️ Drivers Most Hurt by 2026 Regulations:")
for driver, impact in sorted_drivers[-5:]:
    print(f"   {impact:+.3f} positions: {driver}")


📊 2026 REGULATION IMPACT SUMMARY

🏎️ Overall Position Impact:
   - Average change: -0.036 positions
   - Standard deviation: 0.132
   - Max improvement: 0.350
   - Max regression: -0.921
   - Drivers improved: 799 (43.5%)
   - Drivers regressed: 1038 (56.5%)

🏆 Drivers Most Helped by 2026 Regulations:
   -0.014 positions: Pierre Gasly
   -0.020 positions: Fernando Alonso
   -0.023 positions: Oscar Piastri
   -0.023 positions: Max Verstappen
   -0.024 positions: Gabriel Bortoleto

⚠️ Drivers Most Hurt by 2026 Regulations:
   -0.059 positions: Lewis Hamilton
   -0.061 positions: Sebastian Vettel
   -0.063 positions: Franco Colapinto
   -0.077 positions: Nyck De Vries
   -0.090 positions: Oliver Bearman


In [22]:
# Final summary
print("\n" + "="*60)
print("✅ ANALYSIS COMPLETE")
print("="*60)

print(f"\n📁 Outputs saved to: {output_dir}")
print(f"   - monte_carlo_results.json")

print(f"\n🎯 Model Performance:")
print(f"   - MAE: {mae:.2f} positions")
print(f"   - Races simulated: {len(simulation_results)}")
print(f"   - Simulations per race: {config.n_simulations}")

print(f"\n📊 Key Findings:")
print(f"   - 2026 regulations have a {abs(all_changes.mean()):.3f} position average impact")
print(f"   - High-speed tracks benefit most from active aero")
print(f"   - Street circuits may see reduced overtaking")


✅ ANALYSIS COMPLETE

📁 Outputs saved to: e:\5thsem\AIML\f1-2026-simulator\outputs
   - monte_carlo_results.json

🎯 Model Performance:
   - MAE: 0.34 positions
   - Races simulated: 92
   - Simulations per race: 2000

📊 Key Findings:
   - 2026 regulations have a 0.036 position average impact
   - High-speed tracks benefit most from active aero
   - Street circuits may see reduced overtaking


## 📤 Generate JSON Outputs for Frontend

Export regulation impact data in 5 JSON formats for frontend visualization.

In [27]:
# =============================================================================
# GENERATE JSON OUTPUTS FOR FRONTEND VISUALIZATION
# =============================================================================

print("=" * 70)
print("🎯 GENERATING JSON OUTPUTS FOR FRONTEND VISUALIZATION")
print("=" * 70)

# Import the JSON exporter
try:
    from src.json_exporter import export_all_jsons
    
    # Export all JSON files
    json_files = export_all_jsons(
        results=simulation_results,  # Monte Carlo results from previous section
        model_mae=mae,               # Model MAE from training section
        output_dir=project_root / "outputs"
    )
    
    print(f"\n✅ SUCCESS! Generated {len(json_files)} JSON files")
    print(f"\n📂 Output location: {project_root / 'outputs' / 'json'}")
    print("\n📋 Generated files:")
    for f in json_files:
        print(f"   - {f.name}")
    
    print("\n🎉 All JSON exports complete!")
    print("\nYou can now use these files for frontend visualization.")
    
except ImportError as e:
    print(f"\n❌ Import Error: {e}")
    print("\nMake sure src/json_exporter.py exists and src/ is in Python path.")
    print("Try restarting the notebook kernel.")
    
except NameError as e:
    print(f"\n❌ Name Error: {e}")
    print("\nMake sure you've run all previous cells first.")
    print("Required variables: simulation_results, mae, project_root")
    
except Exception as e:
    print(f"\n❌ Error generating JSONs: {e}")
    import traceback
    traceback.print_exc()

🎯 GENERATING JSON OUTPUTS FOR FRONTEND VISUALIZATION
📤 Exporting JSON outputs...
  ✅ Exported driving styles analysis
  ✅ Exported regulation factors breakdown
  ✅ Exported overtaking analysis
  ✅ Exported uncertainty analysis
  ⚠️ Skipped 2022_R01: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R02: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R03: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R04: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R05: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R06: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R07: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R08: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R09: Object of type bool_ is not JSON serializable
  ⚠️ Skipped 2022_R10: Object of type bool_ is not JSON serializable
  ✅ Exported 0 track sector analyses

✅ Total: 4 JSON files created in e:\5thsem\AI